In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
X = np.array(
    [['youth', 'high', 'no', 'fair'],
    ['youth', 'high', 'no', 'excellent'],
    ['middle_aged', 'high', 'no', 'fair'],
    ['senior', 'medium', 'no', 'fair'],
    ['senior', 'low', 'yes', 'fair'],
    ['senior', 'low', 'yes', 'excellent'],
    ['middle_aged', 'low', 'yes', 'excellent'],
    ['youth', 'medium', 'no', 'fair'],
    ['youth', 'low', 'yes', 'fair'],
    ['senior', 'medium', 'yes', 'fair'],
    ['youth', 'medium', 'yes', 'excellent'],
    ['middle_aged', 'medium', 'no', 'excellent'],
    ['middle_aged', 'high', 'yes', 'fair'],
    ['senior', 'medium', 'no', 'excellent']]
)

Y = np.array(['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no'])

In [68]:
def create_node(prop_name, left_value, right_value, left_node, right_node, gini_score, belongs_to_class = None):
  return {
      'prop_name': prop_name,
      'left_value': left_value,
      'right_value': right_value,
      'left_node': left_node,
      'right_node': right_node,
      'gini_score': gini_score,
      'belongs_to_class': belongs_to_class
  }

def calculate_gini_index(Y):
  classes = np.unique(Y)
  result = 1
  for c in classes:
    p = len(Y[Y == c]) / len(Y)
    result -= p**2
  return result

def split_data(X, Y, node):
  chosen_X = None
  min_gini = 1
  left_X_class = None
  right_X_class = None
  X_t = X.transpose()
  for i in range(len(X_t)):
    X_i = X_t[i]
    classes = np.unique(X_i)
    local_min = 1
    local_left_class = None
    local_right_class = None
    print(X_i, classes)
    print("-----------")

    for c in classes:
      left_Y = Y[X_i != c]
      right_Y = Y[X_i == c]
      left_gini = calculate_gini_index(left_Y)
      right_gini = calculate_gini_index(right_Y)
      local_gini = (len(left_Y) / len(Y)) * left_gini + (len(right_Y) / len(Y)) * right_gini
      if local_gini < local_min:
        local_min = local_gini
        print("aaaaa", X_t[i, X_i != c], X_i, c)
        local_left_class = np.unique(X_t[i, X_i != c])
        local_right_class = [c]

    if local_min < min_gini:
      min_gini = local_min
      chosen_X = i
      left_X_class = local_left_class
      right_X_class = local_right_class

  # these are for when Y is already clear for some branch
  left_class = None
  right_class = None
  if min_gini == 0:
    unique_Y_left = np.unique(Y[X_t[chosen_X] != right_X_class[0]])
    unique_Y_right = np.unique(Y[X_t[chosen_X] == right_X_class[0]])

    left_class = unique_Y_left[0] if len(unique_Y_left) == 1 else None
    right_class = unique_Y_right[0] if len(unique_Y_right) == 1 else None


  return min_gini, chosen_X, left_X_class, right_X_class, left_class, right_class

safeguard = 0
def create_tree(X, Y, root = create_node(None, None, None, None, None, None, None)):
  global safeguard
  safeguard += 1
  if safeguard > 20:
    return None
  if root['belongs_to_class'] != None or X.size == 0:
    print("WTFFFFFFFFFFFFFF")
    return None

  min_gini, chosen_X, left_X_class, right_X_class, left_class, right_class = split_data(X, Y, root)

  print(min_gini, chosen_X, left_X_class, right_X_class, left_class, right_class)

  if left_class != None:
    #prune X and Y from left_X_class
    mask = np.isin(X[:, chosen_X], left_X_class)
    X = X[~mask]
    Y = Y[~mask]


  if right_class != None:
    #prune X and Y from left_X_class
    mask = np.isin(X[:, chosen_X], right_X_class)
    X = X[~mask]
    Y = Y[~mask]


  print("sssssssssss", X, len(X))
  if chosen_X != None:
    print("deleting", X, chosen_X)
    X = np.delete(X, chosen_X, 1)


  root['left_node'] = create_tree(X, Y, root)
  root['right_node'] = create_tree(X, Y, root)
  root['gini_score'] = min_gini
  # root['chosen_X'] = chosen_X
  root['left_X_class'] = left_X_class
  root['right_X_class'] = right_X_class


  return create_node(None, None, None, None, None, None, None)

In [69]:
root = create_node(None, None, None, None, None, None, None)
create_tree(X, Y, root)

['youth' 'youth' 'middle_aged' 'senior' 'senior' 'senior' 'middle_aged'
 'youth' 'youth' 'senior' 'youth' 'middle_aged' 'middle_aged' 'senior'] ['middle_aged' 'senior' 'youth']
-----------
aaaaa ['youth' 'youth' 'senior' 'senior' 'senior' 'youth' 'youth' 'senior'
 'youth' 'senior'] ['youth' 'youth' 'middle_aged' 'senior' 'senior' 'senior' 'middle_aged'
 'youth' 'youth' 'senior' 'youth' 'middle_aged' 'middle_aged' 'senior'] middle_aged
['high' 'high' 'high' 'medium' 'low' 'low' 'low' 'medium' 'low' 'medium'
 'medium' 'medium' 'high' 'medium'] ['high' 'low' 'medium']
-----------
aaaaa ['medium' 'low' 'low' 'low' 'medium' 'low' 'medium' 'medium' 'medium'
 'medium'] ['high' 'high' 'high' 'medium' 'low' 'low' 'low' 'medium' 'low' 'medium'
 'medium' 'medium' 'high' 'medium'] high
['no' 'no' 'no' 'no' 'yes' 'yes' 'yes' 'no' 'yes' 'yes' 'yes' 'no' 'yes'
 'no'] ['no' 'yes']
-----------
aaaaa ['yes' 'yes' 'yes' 'yes' 'yes' 'yes' 'yes'] ['no' 'no' 'no' 'no' 'yes' 'yes' 'yes' 'no' 'yes' 'yes' 'yes

{'prop_name': None,
 'left_value': None,
 'right_value': None,
 'left_node': None,
 'right_node': None,
 'gini_score': None,
 'belongs_to_class': None}

In [70]:
print(root)

{'prop_name': None, 'left_value': None, 'right_value': None, 'left_node': {'prop_name': None, 'left_value': None, 'right_value': None, 'left_node': None, 'right_node': None, 'gini_score': None, 'belongs_to_class': None}, 'right_node': {'prop_name': None, 'left_value': None, 'right_value': None, 'left_node': None, 'right_node': None, 'gini_score': None, 'belongs_to_class': None}, 'gini_score': 0.35714285714285715, 'belongs_to_class': None, 'left_X_class': array(['senior', 'youth'], dtype='<U11'), 'right_X_class': ['middle_aged']}
